# Clasificación multiple de 

# Librerias

In [1]:
# general
import numpy as np
import pandas as pd
import re
import os
import joblib
import pandas as pd
import random
import scipy.stats as stats
import pylab
import itertools

# funciones
from funciones.data_exploration import explore
from funciones.feature_cleaning import missing_data, outlier, rare_values
from funciones.feature_cleaning import missing_data as ms
from funciones.feature_cleaning import outlier as ot
from funciones.feature_cleaning import rare_values as ra
from funciones.feature_engineering import discretization as dc
from funciones.feature_engineering import discretization, encoding, transformation
from funciones.feature_selection import embedded_method, feature_shuffle, filter_method, hybrid


# visualización
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import warn
from pandas.plotting import scatter_matrix
plt.style.use('seaborn-colorblind')
#from data_exploration import explore


# preprocesamiento
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder


# Aprendizaje No Supervisado
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, FactorAnalysis
from sklearn.cluster import KMeans, SpectralClustering, Birch, BisectingKMeans
from sklearn.manifold import TSNE, Isomap
from sklearn.discriminant_analysis import  LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis


# ML sklearn
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
import xgboost
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier, LGBMModel
from catboost import CatBoostClassifier


# metricas de evaluacion
from sklearn.metrics import make_scorer, confusion_matrix  # matriz de confusion 
from sklearn.metrics import classification_report # reporte de clasificación

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # clasificación binaria
from sklearn.metrics import average_precision_score

from sklearn.metrics import balanced_accuracy_score, auc, roc_curve, roc_auc_score, RocCurveDisplay # clasificación multi-class
from sklearn.metrics import cohen_kappa_score, hinge_loss, matthews_corrcoef, top_k_accuracy_score


# autoML H2O
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator 
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch


# ML pipelines
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics.pairwise import rbf_kernel

# interpretacion
from sklearn import set_config
import shap

C:\Users\Diego\AppData\Local\Temp\ipykernel_13560\591246234.py:29: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Bases de datos

In [4]:
# cargar bases de datos
data = pd.read_excel('data/AbandonoTC.xlsx')
data

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710


In [5]:
data.columns

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'],
      dtype='object')

# Preprocesamiento

## Limpieza de datos

In [9]:
# calcular porcentaje de missing values y NULLs en cada columna
porcentaje_missing = data.isna().mean() * 100
porcentaje_null = data.isna().mean() * 100

# Sort the variables by missing percentage in descending order
porcentaje_missing.sort_values(ascending=False, inplace=True)
porcentaje_null.sort_values(ascending=False, inplace=True)

# ver variables por porcentaje de NA/NULL
print('porcentaje missing NA',porcentaje_missing)
print('porcentaje NULL NULL',porcentaje_null)

# NO hay NA/NULL

porcentaje missing NA CLIENTNUM                                                                                                                             0.0
Attrition_Flag                                                                                                                        0.0
Avg_Utilization_Ratio                                                                                                                 0.0
Total_Ct_Chng_Q4_Q1                                                                                                                   0.0
Total_Trans_Ct                                                                                                                        0.0
Total_Trans_Amt                                                                                                                       0.0
Total_Amt_Chng_Q4_Q1                                                                                                                  0.0
Avg_Open_To_

In [10]:
# extraer tipos de variables
numeric_columns = data.select_dtypes(include=['float16', 'float32','float64'])
integer_columns = data.select_dtypes(include=['int8', 'int16', 'int32', 'int64'])
categorical_columns = data.select_dtypes(include=['object'])

data_num = data.select_dtypes(exclude=['object'])
data_cate = data.select_dtypes(include=['object'])

In [12]:
integer_columns

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct
0,768805383,45,3,39,5,1,3,777,1144,42
1,818770008,49,5,44,6,1,2,864,1291,33
2,713982108,51,3,36,4,1,0,0,1887,20
3,769911858,40,4,34,3,4,1,2517,1171,20
4,709106358,40,3,21,5,1,0,0,816,28
...,...,...,...,...,...,...,...,...,...,...
10122,772366833,50,2,40,3,2,3,1851,15476,117
10123,710638233,41,2,25,4,2,3,2186,8764,69
10124,716506083,44,1,36,5,3,4,0,10291,60
10125,717406983,30,2,36,4,3,3,0,8395,62


In [11]:
numeric_columns

,Credit_Limit,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
0,12691.0,11914.0,1.335,1.625,0.061,0.000093
1,8256.0,7392.0,1.541,3.714,0.105,0.000057
2,3418.0,3418.0,2.594,2.333,0.000,0.000021
3,3313.0,796.0,1.405,2.333,0.760,0.000134
4,4716.0,4716.0,2.175,2.500,0.000,0.000022
...,...,...,...,...,...,...
10122,4003.0,2152.0,0.703,0.857,0.462,0.000191
10123,4277.0,2091.0,0.804,0.683,0.511,0.995270
10124,5409.0,5409.0,0.819,0.818,0.000,0.997880
10125,5281.0,5281.0,0.535,0.722,0.000,0.996710


In [13]:
categorical_columns

,Attrition_Flag,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
0,Existing Customer,M,High School,Married,$60K - $80K,Blue
1,Existing Customer,F,Graduate,Single,Less than $40K,Blue
2,Existing Customer,M,Graduate,Married,$80K - $120K,Blue
3,Existing Customer,F,High School,Unknown,Less than $40K,Blue
4,Existing Customer,M,Uneducated,Married,$60K - $80K,Blue
...,...,...,...,...,...,...
10122,Existing Customer,M,Graduate,Single,$40K - $60K,Blue
10123,Attrited Customer,M,Unknown,Divorced,$40K - $60K,Blue
10124,Attrited Customer,F,High School,Married,Less than $40K,Blue
10125,Attrited Customer,M,Graduate,Unknown,$40K - $60K,Blue


# EDA

# Machine Learning